In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [3]:
import pandas as pd
import numpy as np
from IPython.display import clear_output

def replace_nan_with_next_value(df, column_name):
        if column_name not in df.columns:
            raise ValueError(f"Column '{column_name}' not found in DataFrame")

        col_values = df[column_name].values

        for i in range(len(col_values) - 1):
            if pd.isna(col_values[i]):
                j = i + 1
                while j < len(col_values) and pd.isna(col_values[j]):
                    j += 1
                if j < len(col_values):
                    col_values[i] = col_values[j]

        df[column_name] = col_values
        return df
    
def delete_rows_with_true(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    df = df[df[column_name] != True]
    return df

def delete_rows_with_work_day(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    df[column_name] = df[column_name].astype(str)

    df = df[df[column_name] != 'Work Day']
    
    return df

def replace_categories_with_integers(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    category_to_integer = {
        'Holiday': 1,
        'Event': 2,
        'Additional': 3,
        'Transfer': 4,
        'Bridge': 5,
        'Regional' : 1,
        'Local' : 2,
        'National' : 3
    }

    df[column_name] = df[column_name].replace(category_to_integer)
    return df

def replace_locale_names_with_integers(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    categories = [
        'Ecuador', 'Riobamba', 'Quito', 'Guaranda', 'Latacunga', 'Ambato',
        'Guayaquil', 'Salinas', 'Loja', 'Santa Elena',
        'Santo Domingo de los Tsachilas', 'Quevedo', 'Ibarra', 'Manta',
        'Esmeraldas', 'Cotopaxi', 'El Carmen', 'Santo Domingo', 'Machala',
        'Imbabura', 'Puyo', 'Libertad', 'Cuenca', 'Cayambe'
    ]

    category_to_integer = {category: index+1 for index, category in enumerate(categories)}

    df[column_name] = df[column_name].replace(category_to_integer)
    return df

def merge_categories_with_same_prefix(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    df[column_name] = df[column_name].str.replace(r'[-+]\d+', '').str.strip()
    return df

def replace_description_with_integers(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    categories = [
    "Navidad",
    "Terremoto Manabi",
    "Fundacion de Quito",
    "Dia de la Madre",
    "Carnaval",
    "Fundacion de Guayaquil",
    "Primer dia del ano",
    "Independencia de Latacunga",
    "Independencia de Cuenca",
    "Provincializacion de Santo Domingo",
    "Provincializacion Santa Elena",
    "Independencia de Guaranda",
    "Fundacion de Loja",
    "Independencia de Ambato",
    "Cantonizacion de Quevedo",
    "Provincializacion de Cotopaxi",
    "Cantonizacion de Salinas",
    "Dia de Difuntos",
    "Fundacion de Manta",
    "Cantonizacion de Libertad",
    "Fundacion de Machala",
    "Fundacion de Ibarra",
    "Cantonizacion de Riobamba",
    "Cantonizacion del Puyo",
    "Cantonizacion de Guaranda",
    "Fundacion de Cuenca",
    "Cantonizacion de Latacunga",
    "Provincializacion de Imbabura",
    "Fundacion de Santo Domingo",
    "Cantonizacion de El Carmen",
    "Cantonizacion de Cayambe",
    "Fundacion de Esmeraldas",
    "Fundacion de Riobamba",
    "Fundacion de Ambato",
    "Viernes Santo",
    "Dia del Trabajo",
    "Mundial de futbol Brasil: Octavos de Final",
    "Primer Grito de Independencia",
    "Independencia de Guayaquil",
    "Cyber Monday",
    "Black Friday",
    "Traslado Independencia de Guayaquil",
    "Batalla de Pichincha",
    "Puente Navidad",
    "Mundial de futbol Brasil: Cuartos de Final",
    "Mundial de futbol Brasil: Semifinales",
    "Traslado Primer Grito de Independencia",
    "Traslado Batalla de Pichincha",
    "Puente Primer dia del ano",
    "Traslado Primer dia del ano",
    "Puente Dia de Difuntos",
    "Traslado Fundacion de Guayaquil",
    "Mundial de futbol Brasil: Ecuador-Honduras",
    "Mundial de futbol Brasil: Ecuador-Francia",
    "Inauguracion Mundial de futbol Brasil",
    "Mundial de futbol Brasil: Final",
    "Mundial de futbol Brasil: Tercer y cuarto lugar",
    "Mundial de futbol Brasil: Ecuador-Suiza",
    "Traslado Fundacion de Quito"
    ]

    category_to_integer = {category: index+1 for index, category in enumerate(categories)}

    df[column_name] = df[column_name].replace(category_to_integer)
    return df

counter = 0


def replace_family_names_with_integers(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    categories = [
    "AUTOMOTIVE",
    "HOME APPLIANCES",
    "SCHOOL AND OFFICE SUPPLIES",
    "PRODUCE",
    "PREPARED FOODS",
    "POULTRY",
    "PLAYERS AND ELECTRONICS",
    "PET SUPPLIES",
    "PERSONAL CARE",
    "MEATS",
    "MAGAZINES",
    "LIQUOR,WINE,BEER",
    "LINGERIE",
    "LAWN AND GARDEN",
    "LADIESWEAR",
    "HOME CARE",
    "HOME AND KITCHEN II",
    "BABY CARE",
    "HOME AND KITCHEN I",
    "HARDWARE",
    "GROCERY II",
    "GROCERY I",
    "FROZEN FOODS",
    "EGGS",
    "DELI",
    "DAIRY",
    "CLEANING",
    "CELEBRATION",
    "BREAD/BAKERY",
    "BOOKS",
    "BEVERAGES",
    "BEAUTY",
    "SEAFOOD"
    ]


    category_to_integer = {category: index+1 for index, category in enumerate(categories)}

    df[column_name] = df[column_name].replace(category_to_integer)
    return df

def csv_process(filepath1, filepath2, filepath3, filepath4):

    oil = pd.read_csv(filepath1)
    
    replace_nan_with_next_value(oil, "dcoilwtico")
    
    #print(oil['dcoilwtico'].value_counts())
    
    #print(oil)
    
    holiday = pd.read_csv(filepath2)
    
    holiday = delete_rows_with_true(holiday, "transferred")
    
    holiday = delete_rows_with_work_day(holiday, "type")
    
    holiday = holiday.drop('transferred', axis=1)
    
    holiday = replace_categories_with_integers(holiday, "type")
    
    holiday = replace_categories_with_integers(holiday, "locale")
    
    holiday = replace_locale_names_with_integers(holiday, "locale_name")
    
    holiday = merge_categories_with_same_prefix(holiday, "description")
    
    holiday = replace_description_with_integers(holiday, "description")
    
    train = pd.read_csv(filepath3)
    
    transactions = pd.read_csv(filepath4)
    
    #train = train.head()
    #train = train.tail()
    
    #transactions = transactions.head(100000)
    #transactions = transactions.tail(20)
    
    #holiday = holiday.head(1000)
    #holiday = holiday.tail(20)
    
    #oil = oil.head(1000)
    #oil = oil.tail(20)
    
    #print("done")
    
    #print(train.info())
    #print(transactions.info())
    
    def drop_before_jan_first_2013(df):
        # Convert the 'date' column to a datetime object
        df['date'] = pd.to_datetime(df['date'])

        # Select rows after January 1st, 2013
        df = df.loc[df['date'] >= '2013-01-01']

        # Reset the index of the resulting dataframe
        df = df.reset_index(drop=True)

        # Return the resulting dataframe
        return df
    
    train = drop_before_jan_first_2013(train)
    
    transactions = drop_before_jan_first_2013(transactions)
    
    holiday = drop_before_jan_first_2013(holiday)
    
    oil = drop_before_jan_first_2013(oil)
    
    def drop_after_august_fifteenth_2017(df):
        # Convert the 'date' column to a datetime object
        df['date'] = pd.to_datetime(df['date'])

        # Select rows after January 1st, 2013
        df = df.loc[df['date'] <= '2017-08-15']

        # Reset the index of the resulting dataframe
        df = df.reset_index(drop=True)

        # Return the resulting dataframe
        return df

    train = drop_after_august_fifteenth_2017(train)
    
    transactions = drop_after_august_fifteenth_2017(transactions)
    
    holiday = drop_after_august_fifteenth_2017(holiday)
    
    oil = drop_after_august_fifteenth_2017(oil)
    
    
    
    #train = train.merge(transactions, on=['store_nbr', 'date'], how='left')
    
    train = replace_family_names_with_integers(train, "family")
    
    
    #print(train)
    #print(holiday.info())
    #print(holiday['date'].value_counts())
    holiday[['type', 'locale', 'locale_name', 'description']] = holiday[['type', 'locale', 'locale_name', 'description']].astype(str)

    # Define a custom function to join the values
    def join_values(column):
        return ''.join(column)

    # Group by date and apply the custom function to each column
    holiday_agg = holiday.groupby('date', as_index=False).agg({
        'type': join_values,
        'locale': join_values,
        'locale_name': join_values,
        'description': join_values
    })
    
    
    
    for i in ['type', 'locale', 'locale_name', 'description']:
        holiday_agg[i] = holiday_agg[i].astype(str).str.replace('[^0-9]', '', regex=True)
    
        # Convert the column to int64
        holiday_agg[i] = pd.to_numeric(holiday_agg[i], errors='coerce').fillna(0).astype(int)
    

    
    date_range = pd.date_range(start=oil['date'].min(), end=oil['date'].max())
    complete_dates = pd.DataFrame({'date': date_range})

    # Merge 'complete_dates' with the 'oil' DataFrame using a left join
    oil_filled = complete_dates.merge(oil, on=['date'], how='left')

    # Fill missing values in the 'dcoilwtico' column with the previous available value
    oil_filled['dcoilwtico'].fillna(method='bfill', inplace=True)

    
    train = train.fillna(0)
   

    return train, oil_filled, holiday_agg, transactions
    
train, oil, holiday, transactions  = csv_process("/kaggle/input/store-sales-time-series-forecasting/oil.csv",
            "/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv",
            "/kaggle/input/store-sales-time-series-forecasting/train.csv",
            "/kaggle/input/store-sales-time-series-forecasting/transactions.csv")

unique_store_numbers = train['store_nbr'].unique()
train_store_dataframes = {}
transaction_store_dataframes = {}
promotion_store_dataframes = {}

for store_number in unique_store_numbers:
    train_store_dataframe = train[train['store_nbr'] == store_number]
    train_store_dataframes[store_number] = train_store_dataframe
    
for store_number in unique_store_numbers:
    transaction_store_dataframe = transactions[transactions['store_nbr'] == store_number]
    transaction_store_dataframes[store_number] = transaction_store_dataframe

for store_number in unique_store_numbers:
    promotion_store_dataframe = train[train['store_nbr'] == store_number]
    promotion_store_dataframes[store_number] = promotion_store_dataframe

    
for store_number in unique_store_numbers:
    train_df = train_store_dataframes[store_number]

    pivot_train_df = train_df.pivot_table(index='date', columns='family', values='sales', fill_value=0)

    # Reset the index to make 'date' a column again
    pivot_train_df.reset_index(inplace=True)

    # Update the dictionary with the modified DataFrame
    train_store_dataframes[store_number] = pivot_train_df

    
for store_number in unique_store_numbers:
    promotion_df = promotion_store_dataframes[store_number]

    pivot_promotion_df = train_df.pivot_table(index='date', columns='family', values='onpromotion', fill_value=0)

    # Reset the index to make 'date' a column again
    pivot_promotion_df.reset_index(inplace=True)

    # Update the dictionary with the modified DataFrame
    promotion_store_dataframes[store_number] = pivot_promotion_df


print(promotion_store_dataframes[1])




print("done")



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:78: FutureWarning: The default value of regex will change from True to False in a future version.


family       date  1  2   3    4  5   6  7  8   9  ...  24  25  26  27  28  \
0      2013-01-01  0  0   0    0  0   0  0  0   0  ...   0   0   0   0   0   
1      2013-01-02  0  0   0    0  0   0  0  0   0  ...   0   0   0   0   0   
2      2013-01-03  0  0   0    0  0   0  0  0   0  ...   0   0   0   0   0   
3      2013-01-04  0  0   0    0  0   0  0  0   0  ...   0   0   0   0   0   
4      2013-01-05  0  0   0    0  0   0  0  0   0  ...   0   0   0   0   0   
...           ... .. ..  ..  ... ..  .. .. ..  ..  ...  ..  ..  ..  ..  ..   
1679   2017-08-11  0  0   7    6  3  22  0  0  11  ...   1  68  18  24   0   
1680   2017-08-12  0  0  10    7  1   0  0  0   7  ...   1   8  15  25   0   
1681   2017-08-13  0  0   8    7  1   0  0  0   9  ...   1   7  19  22   0   
1682   2017-08-14  0  0  11    7  0   0  0  0  10  ...  18   8  18  23   0   
1683   2017-08-15  0  0   8  148  1   0  0  0  11  ...   1   7  19  25   0   

family  29  30  31  32  33  
0        0   0   0   0   0  
1    

In [4]:


for store_number in unique_store_numbers:
    transactions_df = transaction_store_dataframes[store_number]
    sales_df = train_store_dataframes[store_number]
    promotion_df = promotion_store_dataframes[store_number]

    
    date_range = pd.date_range(start=sales_df['date'].min(), end=sales_df['date'].max())
    complete_dates = pd.DataFrame({'date': date_range})

    transactions_df = complete_dates.merge(transactions_df, on=['date'], how='left')
    transactions_df = transactions_df.fillna(0)

    sales_df = complete_dates.merge(sales_df, on=['date'], how='left')
    sales_df = sales_df.fillna(0)
    
    promotion_df = complete_dates.merge(promotion_df, on=['date'], how='left')
    promotion_df = promotion_df.fillna(0)

    date_range = pd.date_range(start=transactions_df['date'].min(), end=transactions_df['date'].max())
    complete_dates = pd.DataFrame({'date': date_range})

    transactions_df = complete_dates.merge(transactions_df, on=['date'], how='left')
    transactions_df = transactions_df.fillna(0)

    sales_df = complete_dates.merge(sales_df, on=['date'], how='left')
    sales_df = sales_df.fillna(0)
    
    promotion_df = complete_dates.merge(promotion_df, on=['date'], how='left')
    promotion_df = promotion_df.fillna(0)

    # Update the dictionaries with the modified DataFrames
    transaction_store_dataframes[store_number] = transactions_df
    train_store_dataframes[store_number] = sales_df
    promotion_store_dataframes[store_number] = promotion_df

    
    
#Sales numbers by store by date
trainst1 = train_store_dataframes[1]
trainst2 = train_store_dataframes[2]
trainst3 = train_store_dataframes[3]
trainst4 = train_store_dataframes[4]
trainst5 = train_store_dataframes[5]
trainst6 = train_store_dataframes[6]
trainst7 = train_store_dataframes[7]
trainst8 = train_store_dataframes[8]
trainst9 = train_store_dataframes[9]
trainst10 = train_store_dataframes[10]
trainst11 = train_store_dataframes[11]
trainst12 = train_store_dataframes[12]
trainst13 = train_store_dataframes[13]
trainst14 = train_store_dataframes[14]
trainst15 = train_store_dataframes[15]
trainst16 = train_store_dataframes[16]
trainst17 = train_store_dataframes[17]
trainst18 = train_store_dataframes[18]
trainst19 = train_store_dataframes[19]
trainst20 = train_store_dataframes[20]
trainst21 = train_store_dataframes[21]
trainst22 = train_store_dataframes[22]
trainst23 = train_store_dataframes[23]
trainst24 = train_store_dataframes[24]
trainst25 = train_store_dataframes[25]
trainst26 = train_store_dataframes[26]
trainst27 = train_store_dataframes[27]
trainst28 = train_store_dataframes[28]
trainst29 = train_store_dataframes[29]
trainst30 = train_store_dataframes[30]
trainst31 = train_store_dataframes[31]
trainst32 = train_store_dataframes[32]
trainst33 = train_store_dataframes[33]
trainst34 = train_store_dataframes[34]
trainst35 = train_store_dataframes[35]
trainst36 = train_store_dataframes[36]
trainst37 = train_store_dataframes[37]
trainst38 = train_store_dataframes[38]
trainst39 = train_store_dataframes[39]
trainst40 = train_store_dataframes[40]
trainst41 = train_store_dataframes[41]
trainst42 = train_store_dataframes[42]
trainst43 = train_store_dataframes[43]
trainst44 = train_store_dataframes[44]
trainst45 = train_store_dataframes[45]
trainst46 = train_store_dataframes[46]
trainst47 = train_store_dataframes[47]
trainst48 = train_store_dataframes[48]
trainst49 = train_store_dataframes[49]
trainst50 = train_store_dataframes[50]
trainst51 = train_store_dataframes[51]
trainst52 = train_store_dataframes[52]
trainst53 = train_store_dataframes[53]
trainst54 = train_store_dataframes[54]

#Transaction numbers by store by date
transst1 = transaction_store_dataframes[1]
transst2 = transaction_store_dataframes[2]
transst3 = transaction_store_dataframes[3]
transst4 = transaction_store_dataframes[4]
transst5 = transaction_store_dataframes[5]
transst6 = transaction_store_dataframes[6]
transst7 = transaction_store_dataframes[7]
transst8 = transaction_store_dataframes[8]
transst9 = transaction_store_dataframes[9]
transst10 = transaction_store_dataframes[10]
transst11 = transaction_store_dataframes[11]
transst12 = transaction_store_dataframes[12]
transst13 = transaction_store_dataframes[13]
transst14 = transaction_store_dataframes[14]
transst15 = transaction_store_dataframes[15]
transst16 = transaction_store_dataframes[16]
transst17 = transaction_store_dataframes[17]
transst18 = transaction_store_dataframes[18]
transst19 = transaction_store_dataframes[19]
transst20 = transaction_store_dataframes[20]
transst21 = transaction_store_dataframes[21]
transst22 = transaction_store_dataframes[22]
transst23 = transaction_store_dataframes[23]
transst24 = transaction_store_dataframes[24]
transst25 = transaction_store_dataframes[25]

promost1 = promotion_store_dataframes[1]
promost2 = promotion_store_dataframes[2]
promost3 = promotion_store_dataframes[3]
promost4 = promotion_store_dataframes[4]
promost5 = promotion_store_dataframes[5]
promost6 = promotion_store_dataframes[6]
promost7 = promotion_store_dataframes[7]
promost8 = promotion_store_dataframes[8]
promost9 = promotion_store_dataframes[9]
promost10 = promotion_store_dataframes[10]
promost11 = promotion_store_dataframes[11]
promost12 = promotion_store_dataframes[12]
promost13 = promotion_store_dataframes[13]
promost14 = promotion_store_dataframes[14]
promost15 = promotion_store_dataframes[15]
promost16 = promotion_store_dataframes[16]
promost17 = promotion_store_dataframes[17]
promost18 = promotion_store_dataframes[18]
promost19 = promotion_store_dataframes[19]
promost20 = promotion_store_dataframes[20]
promost21 = promotion_store_dataframes[21]
promost22 = promotion_store_dataframes[22]
promost23 = promotion_store_dataframes[23]
promost24 = promotion_store_dataframes[24]
promost25 = promotion_store_dataframes[25]
promost26 = promotion_store_dataframes[26]
promost27 = promotion_store_dataframes[27]
promost28 = promotion_store_dataframes[28]
promost29 = promotion_store_dataframes[29]
promost30 = promotion_store_dataframes[30]
promost31 = promotion_store_dataframes[31]
promost32 = promotion_store_dataframes[32]
promost33 = promotion_store_dataframes[33]
promost34 = promotion_store_dataframes[34]
promost35 = promotion_store_dataframes[35]
promost36 = promotion_store_dataframes[36]
promost37 = promotion_store_dataframes[37]
promost38 = promotion_store_dataframes[38]
promost39 = promotion_store_dataframes[39]
promost40 = promotion_store_dataframes[40]
promost41 = promotion_store_dataframes[41]
promost42 = promotion_store_dataframes[42]
promost43 = promotion_store_dataframes[43]
promost44 = promotion_store_dataframes[44]
promost45 = promotion_store_dataframes[45]
promost46 = promotion_store_dataframes[46]
promost47 = promotion_store_dataframes[47]
promost48 = promotion_store_dataframes[48]
promost49 = promotion_store_dataframes[49]
promost50 = promotion_store_dataframes[50]
promost51 = promotion_store_dataframes[51]
promost52 = promotion_store_dataframes[52]
promost53 = promotion_store_dataframes[53]
promost54 = promotion_store_dataframes[54]



print(trainst1.iloc[[356,357,358,359,360,361,362]])
print(transst1.iloc[[356,357,358,359,360,361,362]])
#print(transst1['date'].value_counts())

print(len(trainst1))
print(len(transst1))

print(promost1)

print("done")

          date    1    2    3    4        5        6    7    8      9  ...  \
356 2013-12-23  1.0  0.0  0.0  2.0   83.812  775.088  0.0  0.0  146.0  ...   
357 2013-12-24  1.0  0.0  0.0  4.0  129.199  527.220  0.0  0.0   97.0  ...   
358 2013-12-25  0.0  0.0  0.0  0.0    0.000    0.000  0.0  0.0    0.0  ...   
359 2013-12-26  2.0  0.0  0.0  1.0   77.212  287.849  0.0  0.0  161.0  ...   
360 2013-12-27  6.0  1.0  0.0  9.0   61.527  591.612  0.0  0.0  140.0  ...   
361 2013-12-28  4.0  0.0  0.0  3.0   68.848  201.424  0.0  0.0  115.0  ...   
362 2013-12-29  0.0  0.0  0.0  1.0   19.923  103.545  0.0  0.0   46.0  ...   

        24       25      26      27   28          29   30      31   32      33  
356  191.0  168.141  1074.0  1059.0  0.0  373.752000  0.0  1464.0  2.0  31.551  
357  153.0  158.207   952.0   611.0  0.0  238.080000  0.0  1248.0  2.0  28.162  
358    0.0    0.000     0.0     0.0  0.0    0.000000  0.0     0.0  0.0   0.000  
359  133.0  119.881   595.0   690.0  0.0  307.49400

In [5]:
min_date = trainst1['date'].min()
start_date = min_date + pd.DateOffset(days=30)
end_date = trainst1['date'].max()

date_range = pd.date_range(start=start_date, end=end_date)
complete_dates = pd.DataFrame({'date': date_range})
holiday = complete_dates.merge(holiday, on=['date'], how='left')
holiday = holiday.fillna(0)

types = holiday["type"]
types = (types - types.min()) / (types.max() + 1 - types.min())

locale = holiday["locale"]
locale = (locale - locale.min()) / (locale.max() + 1 - locale.min())

locale_name = holiday["locale_name"]
locale_name = (locale_name - locale_name.min()) / (locale_name.max() + 1 - locale_name.min())

description = holiday["description"]
description = (description - description.min()) / (description.max() + 1 - description.min())

num_features = oil

num_features['day_of_week'] = num_features['date'].dt.dayofweek
num_features['month'] = num_features['date'].dt.month
num_features['day_of_month'] = num_features['date'].dt.day
num_features['quarter'] = num_features['date'].dt.quarter

num_features = num_features.iloc[30:]

num_features.reset_index(inplace=True)

num_features = num_features.drop(columns=['date'])

num_features = (num_features - num_features.min()) / (num_features.max() + 1 - num_features.min())

print(num_features)
print(num_features.info())


types = np.array(types)
locale = np.array(locale)
locale_name = np.array(locale_name)
description = np.array(description)
num_features = np.array(num_features)

print("done")

         index  dcoilwtico  day_of_week     month  day_of_month  quarter
0     0.000000    0.836474     0.428571  0.000000      0.967742      0.0
1     0.000603    0.834250     0.571429  0.083333      0.000000      0.0
2     0.001206    0.819618     0.714286  0.083333      0.032258      0.0
3     0.001809    0.819618     0.857143  0.083333      0.064516      0.0
4     0.002413    0.819618     0.000000  0.083333      0.096774      0.0
...        ...         ...          ...       ...           ...      ...
1653  0.996984    0.264778     0.571429  0.583333      0.322581      0.5
1654  0.997587    0.250497     0.714286  0.583333      0.354839      0.5
1655  0.998191    0.250497     0.857143  0.583333      0.387097      0.5
1656  0.998794    0.250497     0.000000  0.583333      0.419355      0.5
1657  0.999397    0.250263     0.142857  0.583333      0.451613      0.5

[1658 rows x 6 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1658 entries, 0 to 1657
Data columns (total 6 col

In [6]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def prepare_data(transactions, sales, window_size):
    x, y = [], []
    for i in range(len(transactions) - window_size):
        x.append(transactions[i:i+window_size])
        y.append(sales.iloc[i+window_size])
    return np.array(x), np.array(y)




input_data = transst1['transactions']
input_data2 = promost1.drop('date', axis=1)
output_data = trainst1.drop('date', axis=1)

print(input_data)
print(output_data)

x,y = prepare_data(input_data2, output_data, 30)

input_data = transst2['transactions']
input_data2 = promost2.drop('date', axis=1)
output_data = trainst2.drop('date', axis=1)

x2,y2 = prepare_data(input_data2, output_data, 30)

input_data = transst3['transactions']
input_data2 = promost3.drop('date', axis=1)
output_data = trainst3.drop('date', axis=1)

x3,y3 = prepare_data(input_data2, output_data, 30)

input_data = transst4['transactions']
input_data2 = promost4.drop('date', axis=1)
output_data = trainst4.drop('date', axis=1)

x4,y4 = prepare_data(input_data2, output_data, 30)

input_data = transst5['transactions']
input_data2 = promost5.drop('date', axis=1)
output_data = trainst5.drop('date', axis=1)

x5,y5 = prepare_data(input_data2, output_data, 30)

input_data = transst6['transactions']
input_data2 = promost6.drop('date', axis=1)
output_data = trainst6.drop('date', axis=1)

x6,y6 = prepare_data(input_data2, output_data, 30)

input_data = transst7['transactions']
input_data2 = promost7.drop('date', axis=1)
output_data = trainst7.drop('date', axis=1)

x7,y7 = prepare_data(input_data2, output_data, 30)

input_data = transst8['transactions']
input_data2 = promost8.drop('date', axis=1)
output_data = trainst8.drop('date', axis=1)

x8,y8 = prepare_data(input_data2, output_data, 30)

input_data = transst9['transactions']
input_data2 = promost9.drop('date', axis=1)
output_data = trainst9.drop('date', axis=1)

x9,y9 = prepare_data(input_data2, output_data, 30)

input_data = transst10['transactions']
input_data2 = promost10.drop('date', axis=1)
output_data = trainst10.drop('date', axis=1)

x10,y10 = prepare_data(input_data2, output_data, 30)

input_data = transst11['transactions']
input_data2 = promost11.drop('date', axis=1)
output_data = trainst11.drop('date', axis=1)

x11,y11 = prepare_data(input_data2, output_data, 30)

input_data = transst12['transactions']
input_data2 = promost12.drop('date', axis=1)
output_data = trainst12.drop('date', axis=1)

x12,y12 = prepare_data(input_data2, output_data, 30)

input_data = transst13['transactions']
input_data2 = promost13.drop('date', axis=1)
output_data = trainst13.drop('date', axis=1)

x13,y13 = prepare_data(input_data2, output_data, 30)

input_data = transst14['transactions']
input_data2 = promost14.drop('date', axis=1)
output_data = trainst14.drop('date', axis=1)

x14,y14 = prepare_data(input_data2, output_data, 30)

input_data = transst15['transactions']
input_data2 = promost15.drop('date', axis=1)
output_data = trainst15.drop('date', axis=1)

x15,y15 = prepare_data(input_data2, output_data, 30)

input_data = transst16['transactions']
input_data2 = promost16.drop('date', axis=1)
output_data = trainst16.drop('date', axis=1)

x16,y16 = prepare_data(input_data2, output_data, 30)

input_data = transst17['transactions']
input_data2 = promost17.drop('date', axis=1)
output_data = trainst17.drop('date', axis=1)

x17,y17 = prepare_data(input_data2, output_data, 30)

input_data = transst18['transactions']
input_data2 = promost18.drop('date', axis=1)
output_data = trainst18.drop('date', axis=1)

x18,y18 = prepare_data(input_data2, output_data, 30)

input_data = transst19['transactions']
input_data2 = promost19.drop('date', axis=1)
output_data = trainst19.drop('date', axis=1)

x19,y19 = prepare_data(input_data2, output_data, 30)

input_data = transst20['transactions']
input_data2 = promost20.drop('date', axis=1)
output_data = trainst20.drop('date', axis=1)

x20,y20 = prepare_data(input_data2, output_data, 30)

input_data = transst21['transactions']
input_data2 = promost21.drop('date', axis=1)
output_data = trainst21.drop('date', axis=1)

x21,y21 = prepare_data(input_data2, output_data, 30)

input_data = transst22['transactions']
input_data2 = promost22.drop('date', axis=1)
output_data = trainst22.drop('date', axis=1)

x22,y22 = prepare_data(input_data2, output_data, 30)

input_data = transst23['transactions']
input_data2 = promost23.drop('date', axis=1)
output_data = trainst23.drop('date', axis=1)

x23,y23 = prepare_data(input_data2, output_data, 30)

input_data = transst24['transactions']
input_data2 = promost24.drop('date', axis=1)
output_data = trainst24.drop('date', axis=1)

x24,y24 = prepare_data(input_data2, output_data, 30)

input_data = transst25['transactions']
input_data2 = promost25.drop('date', axis=1)
output_data = trainst25.drop('date', axis=1)

x25,y25 = prepare_data(input_data2, output_data, 30)

# Prepare data for store 26
input_data2 = promost26.drop('date', axis=1)
output_data = trainst26.drop('date', axis=1)
x26, y26 = prepare_data(input_data2, output_data, 30)

# Prepare data for store 27
input_data27 = promost27.drop('date', axis=1)
output_data27 = trainst27.drop('date', axis=1)
x27, y27 = prepare_data(input_data27, output_data27, 30)

# Prepare data for store 28
input_data28 = promost28.drop('date', axis=1)
output_data28 = trainst28.drop('date', axis=1)
x28, y28 = prepare_data(input_data28, output_data28, 30)

# Prepare data for store 29
input_data29 = promost29.drop('date', axis=1)
output_data29 = trainst29.drop('date', axis=1)
x29, y29 = prepare_data(input_data29, output_data29, 30)

# Prepare data for store 30
input_data30 = promost30.drop('date', axis=1)
output_data30 = trainst30.drop('date', axis=1)
x30, y30 = prepare_data(input_data30, output_data30, 30)

# Prepare data for store 31
input_data31 = promost31.drop('date', axis=1)
output_data31 = trainst31.drop('date', axis=1)
x31, y31 = prepare_data(input_data31, output_data31, 30)

# Prepare data for store 32
input_data32 = promost32.drop('date', axis=1)
output_data32 = trainst32.drop('date', axis=1)
x32, y32 = prepare_data(input_data32, output_data32, 30)

# Prepare data for store 33
input_data33 = promost33.drop('date', axis=1)
output_data33 = trainst33.drop('date', axis=1)
x33, y33 = prepare_data(input_data33, output_data33, 30)

# Prepare data for store 34
input_data34 = promost34.drop('date', axis=1)
output_data34 = trainst34.drop('date', axis=1)
x34, y34 = prepare_data(input_data34, output_data34, 30)

# Prepare data for store 35
input_data35 = promost35.drop('date', axis=1)
output_data35 = trainst35.drop('date', axis=1)
x35, y35 = prepare_data(input_data35, output_data35, 30)

# Prepare data for store 36
input_data36 = promost36.drop('date', axis=1)
output_data36 = trainst36.drop('date', axis=1)
x36, y36 = prepare_data(input_data36, output_data36, 30)

# Prepare data for store 37
input_data37 = promost37.drop('date', axis=1)
output_data37 = trainst37.drop('date', axis=1)
x37, y37 = prepare_data(input_data37, output_data37, 30)

# Prepare data for store 38
input_data38 = promost38.drop('date', axis=1)
output_data38 = trainst38.drop('date', axis=1)
x38, y38 = prepare_data(input_data38, output_data38, 30)

# Prepare data for store 39
input_data39 = promost39.drop('date', axis=1)
output_data39 = trainst39.drop('date', axis=1)
x39, y39 = prepare_data(input_data39, output_data39, 30)

# Prepare data for store 40
input_data40 = promost40.drop('date', axis=1)
output_data40 = trainst40.drop('date', axis=1)
x40, y40 = prepare_data(input_data40, output_data40, 30)

# Prepare data for store 41
input_data41 = promost41.drop('date', axis=1)
output_data41 = trainst41.drop('date', axis=1)
x41, y41 = prepare_data(input_data41, output_data41, 30)

# Prepare data for store 42
input_data42 = promost42.drop('date', axis=1)
output_data42 = trainst42.drop('date', axis=1)
x42, y42 = prepare_data(input_data42, output_data42, 30)

# Prepare data for store 43
input_data43 = promost43.drop('date', axis=1)
output_data43 = trainst43.drop('date', axis=1)
x43, y43 = prepare_data(input_data43, output_data43, 30)

# Prepare data for store 44
input_data44 = promost44.drop('date', axis=1)
output_data44 = trainst44.drop('date', axis=1)
x44, y44 = prepare_data(input_data44, output_data44, 30)

# Prepare data for store 45
input_data45 = promost45.drop('date', axis=1)
output_data45 = trainst45.drop('date', axis=1)
x45, y45 = prepare_data(input_data45, output_data45, 30)

# Prepare data for store 46
input_data46 = promost46.drop('date', axis=1)
output_data46 = trainst46.drop('date', axis=1)
x46, y46 = prepare_data(input_data46, output_data46, 30)

# Prepare data for store 47
input_data47 = promost47.drop('date', axis=1)
output_data47 = trainst47.drop('date', axis=1)
x47, y47 = prepare_data(input_data47, output_data47, 30)

# Prepare data for store 48
input_data48 = promost48.drop('date', axis=1)
output_data48 = trainst48.drop('date', axis=1)
x48, y48 = prepare_data(input_data48, output_data48, 30)

# Prepare data for store 49
input_data49 = promost49.drop('date', axis=1)
output_data49 = trainst49.drop('date', axis=1)
x49, y49 = prepare_data(input_data49, output_data49, 30)

# Prepare data for store 50
input_data50 = promost50.drop('date', axis=1)
output_data50 = trainst50.drop('date', axis=1)
x50, y50 = prepare_data(input_data50, output_data50, 30)

# Prepare data for store 51
input_data51 = promost51.drop('date', axis=1)
output_data51 = trainst51.drop('date', axis=1)
x51, y51 = prepare_data(input_data51, output_data51, 30)

# Prepare data for store 52
input_data52 = promost52.drop('date', axis=1)
output_data52 = trainst52.drop('date', axis=1)
x52, y52 = prepare_data(input_data52, output_data52, 30)

# Prepare data for store 53
input_data53 = promost53.drop('date', axis=1)
output_data53 = trainst53.drop('date', axis=1)
x53, y53 = prepare_data(input_data53, output_data53, 30)

# Prepare data for store 54
input_data54 = promost54.drop('date', axis=1)
output_data54 = trainst54.drop('date', axis=1)
x54, y54 = prepare_data(input_data54, output_data54, 30)



print(x)
print(y)

print("done")

0          0.0
1       2111.0
2       1833.0
3       1863.0
4       1509.0
         ...  
1683     570.0
1684    1004.0
1685     416.0
1686    1733.0
1687    1693.0
Name: transactions, Length: 1688, dtype: float64
       1    2    3         4          5          6     7    8      9   \
0     0.0  0.0  0.0     0.000   0.000000    0.00000   0.0  0.0    0.0   
1     2.0  0.0  0.0     0.000  47.000000  247.29700   0.0  0.0  194.0   
2     3.0  2.0  0.0     0.000  63.000000  187.27800   0.0  0.0  153.0   
3     3.0  0.0  0.0     0.000  67.000000  258.02300   0.0  0.0   88.0   
4     5.0  0.0  0.0     0.000  66.000000  212.33301   0.0  0.0  141.0   
...   ...  ...  ...       ...        ...        ...   ...  ...    ...   
1683  1.0  0.0  0.0  1115.334  24.963001  259.11800   3.0  5.0   53.0   
1684  6.0  0.0  0.0  1762.493  48.058000  217.66400   6.0  3.0  227.0   
1685  1.0  0.0  0.0   986.669  20.346000  115.75800   0.0  2.0   45.0   
1686  1.0  0.0  0.0  2611.755  72.004000  270.04700   6.

In [7]:
import tensorflow as tf
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

import psutil
import IPython.display
from tensorflow.keras.callbacks import Callback

class ClearOutputAndDisplayEpoch(Callback):
    def on_epoch_end(self, epoch, logs=None):
        IPython.display.clear_output(wait=True)  # Clear the Jupyter Notebook output
        metrics_string = ' - '.join([f'{k}: {v:.4f}' for k, v in logs.items()])
        print(f"Epoch {epoch + 1}/{self.params['epochs']} - {metrics_string}")

class MemoryUsageCheck(Callback):
    def __init__(self, memory_limit_gb=29):
        super().__init__()
        self.memory_limit_bytes = memory_limit_gb * (1024 ** 3)

    def on_epoch_end(self, epoch, logs=None):
        mem_info = psutil.virtual_memory()
        used_memory = mem_info.used
        if used_memory > self.memory_limit_bytes:
            print(f"\nStopping training: RAM usage exceeded {self.memory_limit_bytes / (1024 ** 3):.2f} GB")
            self.model.stop_training = True

memory_usage_check = MemoryUsageCheck(memory_limit_gb=29)
"""
class ClearOutputAndDisplayEpoch(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        if epoch > 0:
            IPython.display.clear_output(wait=True)  # Clear the Jupyter Notebook output
        print(f"Epoch {epoch + 1}/{self.params['epochs']}", end='')
        
    def on_epoch_end(self, epoch, logs=None):
        metrics_string = ' - '.join([f'{k}: {v:.4f}' for k, v in logs.items()])
        print(f" - {metrics_string}")
"""

clear_output_progbar_logger = ClearOutputAndDisplayEpoch()


types_input = tf.keras.layers.Input(shape=(1,))
locale_input = tf.keras.layers.Input(shape=(1,))
locale_name_input = tf.keras.layers.Input(shape=(1,))
description_input = tf.keras.layers.Input(shape=(1,))

oil_and_date_input = tf.keras.layers.Input(shape=(6,)) 


types_embedding_layer = tf.keras.layers.Embedding(input_dim=1, output_dim=20, input_length=1)(types_input)
locale_embedding_layer = tf.keras.layers.Embedding(input_dim=1, output_dim=20, input_length=1)(locale_input)
locale_name_embedding_layer = tf.keras.layers.Embedding(input_dim=1, output_dim=20, input_length=1)(locale_name_input)
description_embedding_layer = tf.keras.layers.Embedding(input_dim=1, output_dim=20, input_length=1)(description_input)

types_flatten = tf.keras.layers.Flatten()(types_embedding_layer)
locale_flatten = tf.keras.layers.Flatten()(locale_embedding_layer)
locale_name_flatten = tf.keras.layers.Flatten()(locale_name_embedding_layer)
description_flatten = tf.keras.layers.Flatten()(description_embedding_layer)


concatenated = tf.keras.layers.Concatenate()([types_flatten, locale_flatten, 
                                              locale_name_flatten, description_flatten, oil_and_date_input])


LSTM_nodes = 33
# If i set only the first LSTM to 64 it works but if I change this to 64 it breaks
# It seems that was just straight up a mistake because this works better

densenodes1 = 256
# maybe 512 is good

densenodes = 256

dense_activation_function = "LeakyReLU"

#num_heads = 4

dense_output_activation_function = "relu"

RNN_activation_function = "tanh"

inputs = []
RNN_layers = []
#RNN_2_layers = []
concs = []
dens1 = []
dens2 = []
dens3 = []
outputs = []

# Create input layers for input24 to input54
for i in range(1, 55):
    input_layer = tf.keras.layers.Input(shape=(30, 33), name=f"input{i}")
    RNN_layer = tf.keras.layers.LSTM(LSTM_nodes, activation=RNN_activation_function, name=f"RNN_layer{i}")(input_layer)
    #RNN_2_layer = tf.keras.layers.LSTM(LSTM_nodes, activation=RNN_activation_function, name=f"RNN_2_layer{i}")(RNN_layer)
    conc_layer = tf.keras.layers.Concatenate(name=f"conc{i}")([RNN_layer, concatenated])
    dense1 = tf.keras.layers.Dense(densenodes1, activation=dense_activation_function, name=f"dense{i}1")(conc_layer)
    dense2 = tf.keras.layers.Dense(densenodes, activation=dense_activation_function, name=f"dense{i}2")(dense1)
    dense3 = tf.keras.layers.Dense(densenodes, activation=dense_activation_function, name=f"dense{i}3")(dense2)
    output_layer = tf.keras.layers.Dense(LSTM_nodes, activation=dense_output_activation_function, name=f"output{i}")(dense3)

    # Append the layers to their respective lists
    inputs.append(input_layer)
    RNN_layers.append(RNN_layer)
    #RNN_2_layers.append(RNN_2_layer)
    concs.append(conc_layer)
    dens1.append(dense1)
    dens2.append(dense2)
    dens3.append(dense3)
    outputs.append(output_layer)


rnn_model = tf.keras.Model(inputs= [types_input, locale_input, locale_name_input,
                                   description_input, oil_and_date_input] + inputs, 
                           outputs= outputs)

def RMSLE(y_pred:tf.Tensor, y_true:tf.Tensor) -> tf.float64:
    """
    The Root Mean Squared Log Error (RMSLE) metric for TensorFlow / Keras
        
    :param y_true: The ground truth labels given in the dataset
    :param y_pred: Predicted values
    :return: The RMSLE score
    """
    y_pred = tf.cast(y_pred, tf.float64)
    y_true = tf.cast(y_true, tf.float64) 
    y_pred = tf.nn.relu(y_pred) 
    return tf.sqrt(tf.reduce_mean(tf.math.squared_difference(tf.math.log1p(y_pred), tf.math.log1p(y_true))))

rnn_model.compile(optimizer='adam', loss=RMSLE, metrics=[tf.keras.losses.MeanAbsoluteError()])

#tf.keras.utils.plot_model(rnn_model, to_file='model.png', show_shapes=True)

rnn_model.summary()




def train_multi_input_model(x_list, y_list, model):

    # Split the inputs and outputs into train and validation sets

    x_train_list, x_val_list, y_train_list, y_val_list = [], [], [], []
    for x in x_list:
        x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.05, random_state=42)
        x_train_list.append(x_train)
        x_val_list.append(x_val)
        y_train_list.append(y_train)
        y_val_list.append(y_val)
    
   
    early_stopping = EarlyStopping(monitor='val_loss', patience=25, mode='min')
    # was 25

    
    # Train the model with the training data and validation data
    history = model.fit(
    x=x_train_list,
    y=y_train_list[:54],
    epochs=1000,
    batch_size=256,
        #use 1024 if training needs to be faster and 256 for more accuracy
    validation_data=(x_val_list, y_val_list[:54]),
    callbacks=[early_stopping, clear_output_progbar_logger, memory_usage_check],
    verbose=2
    )

    return history

x_list = [types, locale, locale_name, description, num_features,
               x, x2, x3, x4, x5,
               x6, x7, x8, x9, x10,
               x11, x12, x13, x14, x15,
               x16, x17, x18, x19, x20,
               x21, x22, x23, x24, x25, x26, x27, x28, x29, x30,
                x31, x32, x33, x34, x35,
                x36, x37, x38, x39, x40,
                x41, x42, x43, x44, x45,
                x46, x47, x48, x49, x50,
                x51, x52, x53, x54]
y_list = [y, y2, y3, y4, y5,
               y6, y7, y8, y9, y10,
               y11, y12, y13, y14, y15,
               y16, y17, y18, y19, y20,
               y21, y22, y23, y24, y25,
             y26, y27, y28, y29, y30,
                y31, y32, y33, y34, y35,
                y36, y37, y38, y39, y40,
                y41, y42, y43, y44, y45,
                y46, y47, y48, y49, y50,
                y51, y52, y53, y54]

#print("Before train-validation split:")
#print("x_train_list length:", len(x_list))
#print("x_val_list length:", len(x_list))

#rnn_model.save("store_sales_predictor")

history = train_multi_input_model(x_list, y_list, rnn_model)
"""

rnn_model.fit([types, locale, locale_name, description, num_features,
               x, x2, x3, x4, x5,
               x6, x7, x8, x9, x10,
               x11, x12, x13, x14, x15,
               x16, x17, x18, x19, x20,
               x21, x22, x23, x24, x25], 
              [y, y2, y3, y4, y5,
               y6, y7, y8, y9, y10,
               y11, y12, y13, y14, y15,
               y16, y17, y18, y19, y20,
               y21, y22, y23, y24, y25],
               epochs=1000, batch_size=1024)
#epochs = 100 lstm = 64 dense = 128 batch_size = 1024   loss = 33.57
#epochs = early stopping patience 25 lstm = 64 dense  = 256 batch_size = 256 loss = 30.8
#epochs = early stopping patience 25 lstm = 64 dense  = 256 batch_size = 256 
# dense_activation_function = "LeakyReLU" dense_output_activation_function = "Relu" 
# RNN_activation_function = "tanh" Loss = 22.88
#epochs = early stopping patience 25 lstm = 33 dense  = 256 batch_size = 256 
# dense_activation_function = "LeakyReLU" dense_output_activation_function = "Relu" 
# RNN_activation_function = "tanh" Loss = 21.1055
"""

Epoch 481/1000 - loss: 47.8653 - output1_loss: 1.0882 - output2_loss: 0.7613 - output3_loss: 1.1996 - output4_loss: 0.7420 - output5_loss: 0.7435 - output6_loss: 0.7962 - output7_loss: 0.7396 - output8_loss: 0.8763 - output9_loss: 0.6791 - output10_loss: 1.0598 - output11_loss: 0.7587 - output12_loss: 0.7535 - output13_loss: 0.7961 - output14_loss: 1.2120 - output15_loss: 0.7381 - output16_loss: 1.1233 - output17_loss: 1.1550 - output18_loss: 1.4109 - output19_loss: 0.7963 - output20_loss: 0.9015 - output21_loss: 0.8694 - output22_loss: 1.2757 - output23_loss: 0.7514 - output24_loss: 0.6273 - output25_loss: 0.7618 - output26_loss: 1.4872 - output27_loss: 0.7659 - output28_loss: 0.7564 - output29_loss: 0.8805 - output30_loss: 0.7553 - output31_loss: 0.6677 - output32_loss: 0.8699 - output33_loss: 0.7404 - output34_loss: 1.1030 - output35_loss: 0.8306 - output36_loss: 0.8169 - output37_loss: 0.7375 - output38_loss: 0.8429 - output39_loss: 1.1331 - output40_loss: 0.7368 - output41_loss: 0

'\n\nrnn_model.fit([types, locale, locale_name, description, num_features,\n               x, x2, x3, x4, x5,\n               x6, x7, x8, x9, x10,\n               x11, x12, x13, x14, x15,\n               x16, x17, x18, x19, x20,\n               x21, x22, x23, x24, x25], \n              [y, y2, y3, y4, y5,\n               y6, y7, y8, y9, y10,\n               y11, y12, y13, y14, y15,\n               y16, y17, y18, y19, y20,\n               y21, y22, y23, y24, y25],\n               epochs=1000, batch_size=1024)\n#epochs = 100 lstm = 64 dense = 128 batch_size = 1024   loss = 33.57\n#epochs = early stopping patience 25 lstm = 64 dense  = 256 batch_size = 256 loss = 30.8\n#epochs = early stopping patience 25 lstm = 64 dense  = 256 batch_size = 256 \n# dense_activation_function = "LeakyReLU" dense_output_activation_function = "Relu" \n# RNN_activation_function = "tanh" Loss = 22.88\n#epochs = early stopping patience 25 lstm = 33 dense  = 256 batch_size = 256 \n# dense_activation_function = "L

In [8]:
import pandas as pd
def replace_nan_with_next_value(df, column_name):
        if column_name not in df.columns:
            raise ValueError(f"Column '{column_name}' not found in DataFrame")

        col_values = df[column_name].values

        for i in range(len(col_values) - 1):
            if pd.isna(col_values[i]):
                j = i + 1
                while j < len(col_values) and pd.isna(col_values[j]):
                    j += 1
                if j < len(col_values):
                    col_values[i] = col_values[j]

        df[column_name] = col_values
        return df
    
def delete_rows_with_true(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    df = df[df[column_name] != True]
    return df

def delete_rows_with_work_day(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    df[column_name] = df[column_name].astype(str)

    df = df[df[column_name] != 'Work Day']
    
    return df

def replace_categories_with_integers(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    category_to_integer = {
        'Holiday': 1,
        'Event': 2,
        'Additional': 3,
        'Transfer': 4,
        'Bridge': 5,
        'Regional' : 1,
        'Local' : 2,
        'National' : 3
    }

    df[column_name] = df[column_name].replace(category_to_integer)
    return df

def replace_locale_names_with_integers(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    categories = [
        'Ecuador', 'Riobamba', 'Quito', 'Guaranda', 'Latacunga', 'Ambato',
        'Guayaquil', 'Salinas', 'Loja', 'Santa Elena',
        'Santo Domingo de los Tsachilas', 'Quevedo', 'Ibarra', 'Manta',
        'Esmeraldas', 'Cotopaxi', 'El Carmen', 'Santo Domingo', 'Machala',
        'Imbabura', 'Puyo', 'Libertad', 'Cuenca', 'Cayambe'
    ]

    category_to_integer = {category: index+1 for index, category in enumerate(categories)}

    df[column_name] = df[column_name].replace(category_to_integer)
    return df

def merge_categories_with_same_prefix(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    df[column_name] = df[column_name].str.replace(r'[-+]\d+', '').str.strip()
    return df

def replace_description_with_integers(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    categories = [
    "Navidad",
    "Terremoto Manabi",
    "Fundacion de Quito",
    "Dia de la Madre",
    "Carnaval",
    "Fundacion de Guayaquil",
    "Primer dia del ano",
    "Independencia de Latacunga",
    "Independencia de Cuenca",
    "Provincializacion de Santo Domingo",
    "Provincializacion Santa Elena",
    "Independencia de Guaranda",
    "Fundacion de Loja",
    "Independencia de Ambato",
    "Cantonizacion de Quevedo",
    "Provincializacion de Cotopaxi",
    "Cantonizacion de Salinas",
    "Dia de Difuntos",
    "Fundacion de Manta",
    "Cantonizacion de Libertad",
    "Fundacion de Machala",
    "Fundacion de Ibarra",
    "Cantonizacion de Riobamba",
    "Cantonizacion del Puyo",
    "Cantonizacion de Guaranda",
    "Fundacion de Cuenca",
    "Cantonizacion de Latacunga",
    "Provincializacion de Imbabura",
    "Fundacion de Santo Domingo",
    "Cantonizacion de El Carmen",
    "Cantonizacion de Cayambe",
    "Fundacion de Esmeraldas",
    "Fundacion de Riobamba",
    "Fundacion de Ambato",
    "Viernes Santo",
    "Dia del Trabajo",
    "Mundial de futbol Brasil: Octavos de Final",
    "Primer Grito de Independencia",
    "Independencia de Guayaquil",
    "Cyber Monday",
    "Black Friday",
    "Traslado Independencia de Guayaquil",
    "Batalla de Pichincha",
    "Puente Navidad",
    "Mundial de futbol Brasil: Cuartos de Final",
    "Mundial de futbol Brasil: Semifinales",
    "Traslado Primer Grito de Independencia",
    "Traslado Batalla de Pichincha",
    "Puente Primer dia del ano",
    "Traslado Primer dia del ano",
    "Puente Dia de Difuntos",
    "Traslado Fundacion de Guayaquil",
    "Mundial de futbol Brasil: Ecuador-Honduras",
    "Mundial de futbol Brasil: Ecuador-Francia",
    "Inauguracion Mundial de futbol Brasil",
    "Mundial de futbol Brasil: Final",
    "Mundial de futbol Brasil: Tercer y cuarto lugar",
    "Mundial de futbol Brasil: Ecuador-Suiza",
    "Traslado Fundacion de Quito"
    ]

    category_to_integer = {category: index+1 for index, category in enumerate(categories)}

    df[column_name] = df[column_name].replace(category_to_integer)
    return df

counter = 0


def replace_family_names_with_integers(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    categories = [
    "AUTOMOTIVE",
    "HOME APPLIANCES",
    "SCHOOL AND OFFICE SUPPLIES",
    "PRODUCE",
    "PREPARED FOODS",
    "POULTRY",
    "PLAYERS AND ELECTRONICS",
    "PET SUPPLIES",
    "PERSONAL CARE",
    "MEATS",
    "MAGAZINES",
    "LIQUOR,WINE,BEER",
    "LINGERIE",
    "LAWN AND GARDEN",
    "LADIESWEAR",
    "HOME CARE",
    "HOME AND KITCHEN II",
    "BABY CARE",
    "HOME AND KITCHEN I",
    "HARDWARE",
    "GROCERY II",
    "GROCERY I",
    "FROZEN FOODS",
    "EGGS",
    "DELI",
    "DAIRY",
    "CLEANING",
    "CELEBRATION",
    "BREAD/BAKERY",
    "BOOKS",
    "BEVERAGES",
    "BEAUTY",
    "SEAFOOD"
    ]


    category_to_integer = {category: index+1 for index, category in enumerate(categories)}

    df[column_name] = df[column_name].replace(category_to_integer)
    return df



def append_rows_after_july_sixteenth_2017(train, supplement):
    # Convert the 'date' column to a datetime object in both DataFrames
    train['date'] = pd.to_datetime(train['date'])
    supplement['date'] = pd.to_datetime(supplement['date'])

    # Filter the 'supplement' DataFrame to only include rows after 2017-07-16
    supplement_filtered = supplement.loc[supplement['date'] > '2017-07-16']

    # Append the filtered 'supplement' DataFrame to the 'train' DataFrame
    train_updated = pd.concat([train, supplement_filtered], ignore_index=True)

    # Sort the updated 'train' DataFrame by date
    train_updated.sort_values(by='date', inplace=True)

    # Reset the index of the resulting dataframe
    train_updated.reset_index(drop=True, inplace=True)

    # Return the updated 'train' DataFrame
    return train_updated


def csv_process(filepath1, filepath2, filepath3, filepath4, filepath5):
    
    supplement = pd.read_csv(filepath5)
    
    supplement = supplement.drop('sales', axis=1)

    oil = pd.read_csv(filepath1)
    
    replace_nan_with_next_value(oil, "dcoilwtico")
    
    #print(oil['dcoilwtico'].value_counts())
    
    #print(oil)
    
    holiday = pd.read_csv(filepath2)
    
    holiday = delete_rows_with_true(holiday, "transferred")
    
    holiday = delete_rows_with_work_day(holiday, "type")
    
    holiday = holiday.drop('transferred', axis=1)
    
    holiday = replace_categories_with_integers(holiday, "type")
    
    holiday = replace_categories_with_integers(holiday, "locale")
    
    holiday = replace_locale_names_with_integers(holiday, "locale_name")
    
    holiday = merge_categories_with_same_prefix(holiday, "description")
    
    holiday = replace_description_with_integers(holiday, "description")
    
    train = pd.read_csv(filepath3)
    
    
    #this might make the difference
    
    
    train = train.sort_values(by=['store_nbr','date'])
    
    
    
    #this might make the difference
    
    
    
    ids = train['id']
    
    train = append_rows_after_july_sixteenth_2017(train, supplement)
    
    transactions = pd.read_csv(filepath4)
    
    #train = train.head()
    #train = train.tail()
    
    #transactions = transactions.head(100000)
    #transactions = transactions.tail(20)
    
    #holiday = holiday.head(1000)
    #holiday = holiday.tail(20)
    
    #oil = oil.head(1000)
    #oil = oil.tail(20)
    
    #print("done")
    
    #print(train.info())
    #print(transactions.info())
    
    
    def drop_before_july_sixteenth_2017(df):
        # Convert the 'date' column to a datetime object
        df['date'] = pd.to_datetime(df['date'])

        # Select rows after January 1st, 2013
        df = df.loc[df['date'] >= '2017-07-16']

        # Reset the index of the resulting dataframe
        df = df.reset_index(drop=True)

        # Return the resulting dataframe
        return df
    
    train = drop_before_july_sixteenth_2017(train)
    
    transactions = drop_before_july_sixteenth_2017(transactions)
    
    holiday = drop_before_july_sixteenth_2017(holiday)
    
    oil = drop_before_july_sixteenth_2017(oil)
    
    def drop_after_august_thirtyfirst_2017(df):
        # Convert the 'date' column to a datetime object
        df['date'] = pd.to_datetime(df['date'])

        # Select rows after January 1st, 2013
        df = df.loc[df['date'] <= '2017-08-31']

        # Reset the index of the resulting dataframe
        df = df.reset_index(drop=True)

        # Return the resulting dataframe
        return df

    train = drop_after_august_thirtyfirst_2017(train)
    
    transactions = drop_after_august_thirtyfirst_2017(transactions)
    
    holiday = drop_after_august_thirtyfirst_2017(holiday)
    
    oil = drop_after_august_thirtyfirst_2017(oil)
    
    print(train['date'].min())
    
    #train = train.merge(transactions, on=['store_nbr', 'date'], how='left')
    
    train = replace_family_names_with_integers(train, "family")
    
    
    
    #print(train)
    #print(holiday.info())
    #print(holiday['date'].value_counts())
    holiday[['type', 'locale', 'locale_name', 'description']] = holiday[['type', 'locale', 'locale_name', 'description']].astype(str)

    # Define a custom function to join the values
    def join_values(column):
        return ''.join(column)

    # Group by date and apply the custom function to each column
    holiday_agg = holiday.groupby('date', as_index=False).agg({
        'type': join_values,
        'locale': join_values,
        'locale_name': join_values,
        'description': join_values
    })
    
    
    
    for i in ['type', 'locale', 'locale_name', 'description']:
        holiday_agg[i] = holiday_agg[i].astype(str).str.replace('[^0-9]', '', regex=True)
    
        # Convert the column to int64
        holiday_agg[i] = pd.to_numeric(holiday_agg[i], errors='coerce').fillna(0).astype(int)
    
    #train = train.merge(holiday_agg, on=["date"], how="left")
    
    #this is what is causing the increase in rows
    #print(train.info())
    #print(train)
    
    
    
    date_range = pd.date_range(start=oil['date'].min(), end=oil['date'].max())
    complete_dates = pd.DataFrame({'date': date_range})

    # Merge 'complete_dates' with the 'oil' DataFrame using a left join
    oil_filled = complete_dates.merge(oil, on=['date'], how='left')

    # Fill missing values in the 'dcoilwtico' column with the previous available value
    oil_filled['dcoilwtico'].fillna(method='bfill', inplace=True)

    
    
    # Now merge the 'train' DataFrame with the 'oil_filled' DataFrame
    #train = train.merge(oil_filled, on=["date"], how="inner")
    
    
    
    #print(holiday['description'].value_counts())
    
    #print(transactions)
    
    #print(train['transactions'])
    
    train = train.fillna(0)
    
    
    
    #print(train.info())
    # look up cartesian product
    

    return train, oil_filled, holiday_agg, transactions, ids
    
train, oil, holiday, transactions, ids  = csv_process("/kaggle/input/store-sales-time-series-forecasting/oil.csv",
            "/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv",
            "/kaggle/input/store-sales-time-series-forecasting/test.csv",
            "/kaggle/input/store-sales-time-series-forecasting/transactions.csv", 
            "/kaggle/input/store-sales-time-series-forecasting/train.csv")

unique_store_numbers = train['store_nbr'].unique()
train_store_dataframes = {}
transaction_store_dataframes = {}
promotion_store_dataframes = {}

""" 
for store_number in unique_store_numbers:
    train_store_dataframe = train[train['store_nbr'] == store_number]
    train_store_dataframes[store_number] = train_store_dataframe
""" 

for store_number in unique_store_numbers:
    transaction_store_dataframe = transactions[transactions['store_nbr'] == store_number]
    transaction_store_dataframes[store_number] = transaction_store_dataframe

for store_number in unique_store_numbers:
    promotion_store_dataframe = train[train['store_nbr'] == store_number]
    promotion_store_dataframes[store_number] = promotion_store_dataframe

"""   
for store_number in unique_store_numbers:
    train_df = train_store_dataframes[store_number]

    pivot_train_df = train_df.pivot_table(index='date', columns='family', values='sales', fill_value=0)

    # Reset the index to make 'date' a column again
    pivot_train_df.reset_index(inplace=True)

    # Update the dictionary with the modified DataFrame
    train_store_dataframes[store_number] = pivot_train_df
"""
    
for store_number in unique_store_numbers:
    promotion_df = promotion_store_dataframes[store_number]

    pivot_promotion_df = promotion_df.pivot_table(index='date', columns='family', values='onpromotion', fill_value=0)

    # Reset the index to make 'date' a column again
    pivot_promotion_df.reset_index(inplace=True)

    # Update the dictionary with the modified DataFrame
    promotion_store_dataframes[store_number] = pivot_promotion_df


#print(promotion_store_dataframes[1])
#print(train)

print(ids)

print("done")


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:75: FutureWarning: The default value of regex will change from True to False in a future version.


2017-07-17 00:00:00
0        3000888
1        3000889
2        3000890
3        3000891
4        3000892
          ...   
28375    3029263
28376    3029264
28377    3029265
28378    3029266
28379    3029267
Name: id, Length: 28512, dtype: int64
done


In [9]:


for store_number in unique_store_numbers:
    promotion_df = promotion_store_dataframes[store_number]

    date_range = pd.date_range(start=promotion_df['date'].min(), end=promotion_df['date'].max())
    complete_dates = pd.DataFrame({'date': date_range})
    
    promotion_df = complete_dates.merge(promotion_df, on=['date'], how='left')
    promotion_df = promotion_df.fillna(0)

    # Update the dictionaries with the modified DataFrames
    promotion_store_dataframes[store_number] = promotion_df

    
"""    
#Sales numbers by store by date
trainst1 = train_store_dataframes[1]
trainst2 = train_store_dataframes[2]
trainst3 = train_store_dataframes[3]
trainst4 = train_store_dataframes[4]
trainst5 = train_store_dataframes[5]
trainst6 = train_store_dataframes[6]
trainst7 = train_store_dataframes[7]
trainst8 = train_store_dataframes[8]
trainst9 = train_store_dataframes[9]
trainst10 = train_store_dataframes[10]
trainst11 = train_store_dataframes[11]
trainst12 = train_store_dataframes[12]
trainst13 = train_store_dataframes[13]
trainst14 = train_store_dataframes[14]
trainst15 = train_store_dataframes[15]
trainst16 = train_store_dataframes[16]
trainst17 = train_store_dataframes[17]
trainst18 = train_store_dataframes[18]
trainst19 = train_store_dataframes[19]
trainst20 = train_store_dataframes[20]
trainst21 = train_store_dataframes[21]
trainst22 = train_store_dataframes[22]
trainst23 = train_store_dataframes[23]
trainst24 = train_store_dataframes[24]
trainst25 = train_store_dataframes[25]


#Transaction numbers by store by date
transst1 = transaction_store_dataframes[1]
transst2 = transaction_store_dataframes[2]
transst3 = transaction_store_dataframes[3]
transst4 = transaction_store_dataframes[4]
transst5 = transaction_store_dataframes[5]
transst6 = transaction_store_dataframes[6]
transst7 = transaction_store_dataframes[7]
transst8 = transaction_store_dataframes[8]
transst9 = transaction_store_dataframes[9]
transst10 = transaction_store_dataframes[10]
transst11 = transaction_store_dataframes[11]
transst12 = transaction_store_dataframes[12]
transst13 = transaction_store_dataframes[13]
transst14 = transaction_store_dataframes[14]
transst15 = transaction_store_dataframes[15]
transst16 = transaction_store_dataframes[16]
transst17 = transaction_store_dataframes[17]
transst18 = transaction_store_dataframes[18]
transst19 = transaction_store_dataframes[19]
transst20 = transaction_store_dataframes[20]
transst21 = transaction_store_dataframes[21]
transst22 = transaction_store_dataframes[22]
transst23 = transaction_store_dataframes[23]
transst24 = transaction_store_dataframes[24]
transst25 = transaction_store_dataframes[25]

"""

#Promotions by family and store number
#Promotions by family and store number
#Promotions by family and store number
#Promotions by family and store number
#Promotions by family and store number
#Promotions by family and store number

promost1 = promotion_store_dataframes[1]
promost2 = promotion_store_dataframes[2]
promost3 = promotion_store_dataframes[3]
promost4 = promotion_store_dataframes[4]
promost5 = promotion_store_dataframes[5]
promost6 = promotion_store_dataframes[6]
promost7 = promotion_store_dataframes[7]
promost8 = promotion_store_dataframes[8]
promost9 = promotion_store_dataframes[9]
promost10 = promotion_store_dataframes[10]
promost11 = promotion_store_dataframes[11]
promost12 = promotion_store_dataframes[12]
promost13 = promotion_store_dataframes[13]
promost14 = promotion_store_dataframes[14]
promost15 = promotion_store_dataframes[15]
promost16 = promotion_store_dataframes[16]
promost17 = promotion_store_dataframes[17]
promost18 = promotion_store_dataframes[18]
promost19 = promotion_store_dataframes[19]
promost20 = promotion_store_dataframes[20]
promost21 = promotion_store_dataframes[21]
promost22 = promotion_store_dataframes[22]
promost23 = promotion_store_dataframes[23]
promost24 = promotion_store_dataframes[24]
promost25 = promotion_store_dataframes[25]
promost26 = promotion_store_dataframes[26]
promost27 = promotion_store_dataframes[27]
promost28 = promotion_store_dataframes[28]
promost29 = promotion_store_dataframes[29]
promost30 = promotion_store_dataframes[30]
promost31 = promotion_store_dataframes[31]
promost32 = promotion_store_dataframes[32]
promost33 = promotion_store_dataframes[33]
promost34 = promotion_store_dataframes[34]
promost35 = promotion_store_dataframes[35]
promost36 = promotion_store_dataframes[36]
promost37 = promotion_store_dataframes[37]
promost38 = promotion_store_dataframes[38]
promost39 = promotion_store_dataframes[39]
promost40 = promotion_store_dataframes[40]
promost41 = promotion_store_dataframes[41]
promost42 = promotion_store_dataframes[42]
promost43 = promotion_store_dataframes[43]
promost44 = promotion_store_dataframes[44]
promost45 = promotion_store_dataframes[45]
promost46 = promotion_store_dataframes[46]
promost47 = promotion_store_dataframes[47]
promost48 = promotion_store_dataframes[48]
promost49 = promotion_store_dataframes[49]
promost50 = promotion_store_dataframes[50]
promost51 = promotion_store_dataframes[51]
promost52 = promotion_store_dataframes[52]
promost53 = promotion_store_dataframes[53]
promost54 = promotion_store_dataframes[54]


"""
print(trainst1.iloc[[356,357,358,359,360,361,362]])
print(transst1.iloc[[356,357,358,359,360,361,362]])
#print(transst1['date'].value_counts())

print(len(trainst1))
print(len(transst1))
"""

print(promost1)

print("done")

         date  1  2   3    4  5   6  7  8   9  ...  24  25  26  27  28  29  \
0  2017-07-17  0  0   0    3  0   0  0  0   4  ...   0  13  17  15   0   0   
1  2017-07-18  0  0   0    3  0   0  0  0   5  ...   0  13  18  19   0   0   
2  2017-07-19  0  0   0  210  0   0  0  0   4  ...   0  11  19  22   0   0   
3  2017-07-20  0  0   0   10  0   0  0  0   5  ...   0  10  17  19   0   0   
4  2017-07-21  0  0   0    9  0  38  0  0   4  ...  29  45  18  17   0   0   
5  2017-07-22  0  0   0   10  0   0  0  0   5  ...   0  11  19  15   0   0   
6  2017-07-23  0  0   0    9  0   0  0  0   5  ...   0   4  16  10   0   0   
7  2017-07-24  0  0   0   11  0   0  0  0   5  ...   0  10  18  12   0   0   
8  2017-07-25  0  0   0   10  0   0  0  0   6  ...   0  11  18  10   0   0   
9  2017-07-26  0  0   0  211  0   0  0  0   4  ...   0  15  17  11   0   0   
10 2017-07-27  0  0   0   13  0   0  0  0   6  ...   0   8   8  15   0   0   
11 2017-07-28  0  0   0   13  0  37  0  0   6  ...  25  54   7  

In [10]:
min_date = promost1['date'].min()
start_date = min_date + pd.DateOffset(days=30)
end_date = promost1['date'].max()

date_range = pd.date_range(start=start_date, end=end_date)
complete_dates = pd.DataFrame({'date': date_range})
holiday = complete_dates.merge(holiday, on=['date'], how='left')
holiday = holiday.fillna(0)

types = holiday["type"]
types = (types - types.min()) / (types.max() + 1 - types.min())

locale = holiday["locale"]
locale = (locale - locale.min()) / (locale.max() + 1 - locale.min())

locale_name = holiday["locale_name"]
locale_name = (locale_name - locale_name.min()) / (locale_name.max() + 1 - locale_name.min())

description = holiday["description"]
description = (description - description.min()) / (description.max() + 1 - description.min())

num_features = oil

num_features['day_of_week'] = num_features['date'].dt.dayofweek
num_features['month'] = num_features['date'].dt.month
num_features['day_of_month'] = num_features['date'].dt.day
num_features['quarter'] = num_features['date'].dt.quarter

num_features = num_features.iloc[30:]

num_features.reset_index(inplace=True)

num_features = num_features.drop(columns=['date'])

num_features = (num_features - num_features.min()) / (num_features.max() + 1 - num_features.min())

print(num_features)
print(num_features.info())


types = np.array(types)
locale = np.array(locale)
locale_name = np.array(locale_name)
description = np.array(description)
num_features = np.array(num_features)

print(num_features)

print("done")

     index  dcoilwtico  day_of_week  month  day_of_month  quarter
0   0.0000    0.231405     0.285714    0.0        0.0000      0.0
1   0.0625    0.305785     0.428571    0.0        0.0625      0.0
2   0.1250    0.724518     0.571429    0.0        0.1250      0.0
3   0.1875    0.393939     0.714286    0.0        0.1875      0.0
4   0.2500    0.393939     0.857143    0.0        0.2500      0.0
5   0.3125    0.393939     0.000000    0.0        0.3125      0.0
6   0.3750    0.465565     0.142857    0.0        0.3750      0.0
7   0.4375    0.685950     0.285714    0.0        0.4375      0.0
8   0.5000    0.352617     0.428571    0.0        0.5000      0.0
9   0.5625    0.465565     0.571429    0.0        0.5625      0.0
10  0.6250    0.121212     0.714286    0.0        0.6250      0.0
11  0.6875    0.121212     0.857143    0.0        0.6875      0.0
12  0.7500    0.121212     0.000000    0.0        0.7500      0.0
13  0.8125    0.137741     0.142857    0.0        0.8125      0.0
14  0.8750

In [11]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def prepare_data(promost, window_size):
    x = []
    for i in range(len(promost) - window_size):
        combined_data = promost[i:i+window_size]
        x.append(combined_data)
    return np.array(x)

input_data2 = promost1.drop('date', axis=1)
x = prepare_data(input_data2, 30)

input_data2 = promost2.drop('date', axis=1)
x2 = prepare_data(input_data2, 30)

input_data2 = promost3.drop('date', axis=1)
x3 = prepare_data(input_data2, 30)

input_data2 = promost4.drop('date', axis=1)
x4 = prepare_data(input_data2, 30)

input_data2 = promost5.drop('date', axis=1)
x5 = prepare_data(input_data2, 30)

input_data2 = promost6.drop('date', axis=1)
x6 = prepare_data(input_data2, 30)

input_data2 = promost7.drop('date', axis=1)
x7 = prepare_data(input_data2, 30)

input_data2 = promost8.drop('date', axis=1)
x8 = prepare_data(input_data2, 30)

input_data2 = promost9.drop('date', axis=1)
x9 = prepare_data(input_data2, 30)

input_data2 = promost10.drop('date', axis=1)
x10 = prepare_data(input_data2, 30)

input_data2 = promost11.drop('date', axis=1)
x11 = prepare_data(input_data2, 30)

input_data2 = promost12.drop('date', axis=1)
x12 = prepare_data(input_data2, 30)

input_data2 = promost13.drop('date', axis=1)
x13 = prepare_data(input_data2, 30)

input_data2 = promost14.drop('date', axis=1)
x14 = prepare_data(input_data2, 30)

input_data2 = promost15.drop('date', axis=1)
x15 = prepare_data(input_data2, 30)

input_data2 = promost16.drop('date', axis=1)
x16 = prepare_data(input_data2, 30)

input_data2 = promost17.drop('date', axis=1)
x17 = prepare_data(input_data2, 30)

input_data2 = promost18.drop('date', axis=1)
x18 = prepare_data(input_data2, 30)

input_data2 = promost19.drop('date', axis=1)
x19 = prepare_data(input_data2, 30)

input_data2 = promost20.drop('date', axis=1)
x20 = prepare_data(input_data2, 30)

input_data2 = promost21.drop('date', axis=1)
x21 = prepare_data(input_data2, 30)

input_data2 = promost22.drop('date', axis=1)
x22 = prepare_data(input_data2, 30)

input_data2 = promost23.drop('date', axis=1)
x23 = prepare_data(input_data2, 30)

input_data2 = promost24.drop('date', axis=1)
x24 = prepare_data(input_data2, 30)

input_data2 = promost25.drop('date', axis=1)
x25 = prepare_data(input_data2, 30)

input_data2 = promost26.drop('date', axis=1)
x26 = prepare_data(input_data2, 30)

input_data2 = promost27.drop('date', axis=1)
x27 = prepare_data(input_data2, 30)

input_data2 = promost28.drop('date', axis=1)
x28 = prepare_data(input_data2, 30)

input_data2 = promost29.drop('date', axis=1)
x29 = prepare_data(input_data2, 30)

input_data2 = promost30.drop('date', axis=1)
x30 = prepare_data(input_data2, 30)

input_data2 = promost31.drop('date', axis=1)
x31 = prepare_data(input_data2, 30)

input_data2 = promost32.drop('date', axis=1)
x32 = prepare_data(input_data2, 30)

input_data2 = promost33.drop('date', axis=1)
x33 = prepare_data(input_data2, 30)

input_data2 = promost34.drop('date', axis=1)
x34 = prepare_data(input_data2, 30)

input_data2 = promost35.drop('date', axis=1)
x35 = prepare_data(input_data2, 30)

input_data2 = promost36.drop('date', axis=1)
x36 = prepare_data(input_data2, 30)

input_data2 = promost37.drop('date', axis=1)
x37 = prepare_data(input_data2, 30)

input_data2 = promost38.drop('date', axis=1)
x38 = prepare_data(input_data2, 30)

input_data2 = promost39.drop('date', axis=1)
x39 = prepare_data(input_data2, 30)

input_data2 = promost40.drop('date', axis=1)
x40 = prepare_data(input_data2, 30)

input_data2 = promost41.drop('date', axis=1)
x41 = prepare_data(input_data2, 30)

input_data2 = promost42.drop('date', axis=1)
x42 = prepare_data(input_data2, 30)

input_data2 = promost43.drop('date', axis=1)
x43 = prepare_data(input_data2, 30)

input_data2 = promost44.drop('date', axis=1)
x44 = prepare_data(input_data2, 30)

input_data2 = promost45.drop('date', axis=1)
x45 = prepare_data(input_data2, 30)

input_data2 = promost46.drop('date', axis=1)
x46 = prepare_data(input_data2, 30)

input_data2 = promost47.drop('date', axis=1)
x47 = prepare_data(input_data2, 30)

input_data2 = promost48.drop('date', axis=1)
x48 = prepare_data(input_data2, 30)

input_data2 = promost49.drop('date', axis=1)
x49 = prepare_data(input_data2, 30)

input_data2 = promost50.drop('date', axis=1)
x50 = prepare_data(input_data2, 30)

input_data2 = promost51.drop('date', axis=1)
x51 = prepare_data(input_data2, 30)

input_data2 = promost52.drop('date', axis=1)
x52 = prepare_data(input_data2, 30)

input_data2 = promost53.drop('date', axis=1)
x53 = prepare_data(input_data2, 30)

input_data2 = promost54.drop('date', axis=1)
x54 = prepare_data(input_data2, 30)




print(x)


print("done")

[[[ 0  0  0 ... 39  0  0]
  [ 0  0  0 ... 42  0  0]
  [ 0  0  0 ... 34  0  0]
  ...
  [ 0  0  0 ...  5  0  0]
  [ 0  0  0 ...  9  1  0]
  [ 0  0  0 ... 11  1  0]]

 [[ 0  0  0 ... 42  0  0]
  [ 0  0  0 ... 34  0  0]
  [ 0  0  0 ... 32  0  0]
  ...
  [ 0  0  0 ...  9  1  0]
  [ 0  0  0 ... 11  1  0]
  [ 0  0 14 ... 20  2  0]]

 [[ 0  0  0 ... 34  0  0]
  [ 0  0  0 ... 32  0  0]
  [ 0  0  0 ... 30  0  6]
  ...
  [ 0  0  0 ... 11  1  0]
  [ 0  0 14 ... 20  2  0]
  [ 0  0  0 ... 17  1  0]]

 ...

 [[ 0  0  0 ... 16  0  0]
  [ 0  0  0 ... 24  0  0]
  [ 0  0  0 ... 26  0  0]
  ...
  [ 0  0  0 ... 32  1  1]
  [ 0  0  0 ... 23  1  0]
  [ 0  0  0 ... 31  1  1]]

 [[ 0  0  0 ... 24  0  0]
  [ 0  0  0 ... 26  0  0]
  [ 0  0  0 ... 25  1  0]
  ...
  [ 0  0  0 ... 23  1  0]
  [ 0  0  0 ... 31  1  1]
  [ 0  0  0 ... 31  1  0]]

 [[ 0  0  0 ... 26  0  0]
  [ 0  0  0 ... 25  1  0]
  [ 0  0  0 ...  7  1  0]
  ...
  [ 0  0  0 ... 31  1  1]
  [ 0  0  0 ... 31  1  0]
  [ 0  0  0 ... 35  0  1]]]
done


In [12]:
print(ids)

predictions = rnn_model.predict([types, locale, locale_name, description, num_features,
               x, x2, x3, x4, x5,
               x6, x7, x8, x9, x10,
               x11, x12, x13, x14, x15,
               x16, x17, x18, x19, x20,
               x21, x22, x23, x24, x25,
            x26, x27, x28, x29, x30,
                x31, x32, x33, x34, x35,
                x36, x37, x38, x39, x40,
                x41, x42, x43, x44, x45,
                x46, x47, x48, x49, x50,
                x51, x52, x53, x54])

predictions = predictions

print(predictions)

prediction_df = pd.DataFrame({'id': ids,
                              'sales': np.squeeze(predictions).flatten()})

print(prediction_df)

prediction_df.to_csv("predictions.csv", index=False)

0        3000888
1        3000889
2        3000890
3        3000891
4        3000892
          ...   
28375    3029263
28376    3029264
28377    3029265
28378    3029266
28379    3029267
Name: id, Length: 28512, dtype: int64
1/1 [==============================] - 19s 19s/step
[array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.40921967e+02,
        2.95501556e+01, 1.05944344e+02, 8.21373984e-02, 0.00000000e+00,
        4.84081001e+01, 1.01550522e+02, 0.00000000e+00, 3.33934212e+01,
        0.00000000e+00, 5.57240963e+00, 4.91218281e+00, 4.30027733e+01,
        5.74804783e-01, 0.00000000e+00, 4.04346561e+00, 0.00000000e+00,
        1.13186131e+01, 7.45866516e+02, 0.00000000e+00, 5.03217239e+01,
        4.24430199e+01, 2.28471359e+02, 2.19997330e+02, 0.00000000e+00,
        1.16949791e+02, 0.00000000e+00, 5.77226257e+02, 0.00000000e+00,
        9.64849567e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.90820935e+03,
        8.61390762e+01, 3.13898346e+02, 8.1002187